In [1]:
import pandas as pd
import glob
import re
import string

In [2]:
filelist = glob.glob("csvs/*.csv")

In [3]:
dash = '-' * 90

for i in range(len(filelist)):
    if i == 0:
      print(dash)
      print('{:<40s}{:>20s}{:>20s}'.format('Name','Kommentare Bekommen','Kommentare Zahl'))
      print(dash)
    else:
      df = pd.read_csv(filelist[i-1])
      print('{:<40s}{:>10d}{:>25d}'.format(filelist[i-1], len(df['Kommentare']), df['Zahl der Kommentaren'][0]))

------------------------------------------------------------------------------------------
Name                                     Kommentare Bekommen     Kommentare Zahl
------------------------------------------------------------------------------------------
csvs/KRH Klinikum Lehrte.csv                    87                       87
csvs/Sophienklinik GmbH.csv                    116                      116
csvs/Krankenhaus Lindenbrunn.csv                44                       44
csvs/Asklepios Hospital Göttingen.csv           56                       57
csvs/Augenklinik Dr. Hoffmann.csv              108                      108
csvs/Helios Klinikum Hildesheim.csv            305                      307
csvs/Asklepios Hospital Tiefenbrunn.csv         25                       25
csvs/Krankenhaus Marienstift gGmbH.csv          86                       86
csvs/DIAKOVERE Friederikenstift.csv            264                      264
csvs/DIAKOVERE Annastift.csv                    98   

#### Google Maps CSV's kombinieren

In [4]:
def kombineCSVs(filenames):
    df = pd.concat([pd.read_csv(file, encoding='utf-8') for file in filenames])
    return df

In [5]:
maps_df = kombineCSVs([name for name in filelist])

In [6]:
maps_df.head(3)

,Name,AVG - Sterne,Zahl der Kommentaren,Adresse,Sterne,Kommentare,Datum,Likes
0,KRH Klinikum Lehrte,3.3,87,"Manskestraße 22, 31275 Lehrte",4,(Translated by Google) My father was admitted ...,a month ago,0
1,KRH Klinikum Lehrte,3.3,87,"Manskestraße 22, 31275 Lehrte",5,(Translated by Google) Station 4 is great. I c...,a year ago,2
2,KRH Klinikum Lehrte,3.3,87,"Manskestraße 22, 31275 Lehrte",5,(Translated by Google) Everyone was super nice...,a month ago,0


In [7]:
maps_df.reset_index(drop=True, inplace=True)

In [8]:
strin = maps_df['Kommentare'][0]
strin

"(Translated by Google) My father was admitted to the intensive care unit in this hospital. The staff did everything to save him. Thanks alot! You are angels on earth.\nOne star, because the communication wasn't so good and some of the sisters were really unfriendly.\n\n(Original)\nMein Vater ist in diesem KH in die Intensivstation eingeliefert worden. Das Personal hat alles getan um.ihn zu retten. Vielen vielen Dank! Ihr seid Engel auf Erden.\nEinen Stern Abzug, weil die Kommunikation nicht so gut war und manche Schwestern echt unfreundlich waren."

#### Bearbeiten von Kommentaren, um gute Ergebnisse aus der Sentimentalyse zu erhalten

##### Unterscheidung zwischen englischen und deutschen Kommentaren

In [9]:
def split_it(df, column):
    english = []
    deutsch = []
    komment_list = []
    df['Regex'] =  [re.split(r"\(Translated by Google\)", str(x)) for x in df[column]]
    for kommentar in df['Regex']:
        try:
            komment = kommentar[1]
            komment_list.append(komment)
        except:
            komment = 'No Comment'
            komment_list.append(komment)

    df['Regex'] =  komment_list
    
    df['Regex'] =  [re.split(r"\(Original\)", str(x)) for x in df['Regex']]
    
    for sprache in df['Regex']:
        try:
            eng = sprache[0]
            de = sprache[1]
            english.append(eng)
            deutsch.append(de)
        except:
            komment = 'No Comment'
            english.append(komment)
            deutsch.append(komment)
    df.drop('Regex', axis=1, inplace= True)
    df['Englisch'] =  english
    df['Deutsch'] =  deutsch 
    
    return df['Englisch'], df['Deutsch']

In [10]:
maps_df['Englisch'], maps_df['Deutsch'] = split_it(df, 'Kommentare')

In [11]:
google_maps = maps_df.drop('Kommentare', axis=1)

In [12]:
google_maps.head(2)

,Name,AVG - Sterne,Zahl der Kommentaren,Adresse,Sterne,Datum,Likes,Englisch,Deutsch
0,KRH Klinikum Lehrte,3.3,87,"Manskestraße 22, 31275 Lehrte",4,a month ago,0,No Comment,No Comment
1,KRH Klinikum Lehrte,3.3,87,"Manskestraße 22, 31275 Lehrte",5,a year ago,2,"Great hospital, but I'd rather be healthy. Wa...","\nTolles Krankenhaus, trotzdem bin ich lieber..."


##### Entfernen unerwünschter Zeichen aus Kommentaren

In [13]:
google_maps['Englisch'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
google_maps['Deutsch'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 

In [14]:
google_maps['Deutsch'][0]

'No Comment'

In [15]:
google_maps['Adresse'].value_counts()

Senator-Braun-Allee 33, 31135 Hildesheim              305
Saint-Maur-Platz 1, 31785 Hameln                      275
Humboldtstraße 5, 30169 Hannover                      264
Lützerodestraße 1, 30161 Hannover                     178
Leipziger Str. 24, 38124 Braunschweig                 140
Bischofsholer Damm 160, 30173 Hannover                116
Fuhrberger Str. 8, 30938 Burgwedel                    113
Wolfenbütteler Str. 82, 38102 Braunschweig            108
Anna-von-Borries-Straße 1-7, 30625 Hannover            98
Manskestraße 22, 31275 Lehrte                          87
Helmstedter Str. 35, 38102 Braunschweig                86
Landrat-Beushausen-Straße 26, 31061 Alfeld (Leine)     70
Rosdorfer Weg 70, 37081 Göttingen                      56
Am Lindenbrunn 1, 31863 Coppenbrügge                   44
Tiefenbrunn, 37124 Rosdorf                             25
Name: Adresse, dtype: int64

In [16]:
google_maps.head(2)

,Name,AVG - Sterne,Zahl der Kommentaren,Adresse,Sterne,Datum,Likes,Englisch,Deutsch
0,KRH Klinikum Lehrte,3.3,87,"Manskestraße 22, 31275 Lehrte",4,a month ago,0,No Comment,No Comment
1,KRH Klinikum Lehrte,3.3,87,"Manskestraße 22, 31275 Lehrte",5,a year ago,2,"Great hospital, but I'd rather be healthy. Wa...","Tolles Krankenhaus, trotzdem bin ich lieber g..."


In [ ]:
# yorum aralarındaki /n ve tırnak gibi karakterleri temizle... 

In [ ]:
# bewertung ve maps csv den tek tablo oluştur

In [ ]:
# data typeları düzenle

In [ ]:
# comments ayır

In [ ]:
# ML algorithm entscheiden um eine KlinifyScore zu herausfinden.

In [ ]:
# bewertung sterne int yap... 